In [1]:
import whisper
import os
model = whisper.load_model('large-v2')
DATA_DIR = 'C:/src/interview-summary/data'
SOURCE_FILE = os.path.join(DATA_DIR, '2min.wav')

In [2]:
result = model.transcribe(SOURCE_FILE)
print(result["text"])

c:\src\interview-summary\venv\lib\site-packages\whisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Goedemiddag. Hoi, goedemiddag Angelique. Hoi. Ik heb op dit moment geen beeld. Ik weet niet wat jij nu voor je ziet. Ik zie jou. Ik krijg een zwart beeld bij jou staan. Ja, dat klopt. Omdat mijn vakje nog niet open is. Kijk. Dat kan ik ook niet zien. Dat praat toch wat makkelijker. Ja, precies. Lukt het een beetje zo op de digitale manier? Jawel, jawel. Ik ben opgegroeid in het digitaal, dus. Kijk. Jonge generatie, hè. Kijk, helemaal super. Ja, fijn. Het is niet voor iedereen even makkelijk, dus zo fijn. Nee, dat is waar, dat is waar. Super. Goed dat je er bent. Nou. Ja, en ik had ook al eventjes gekeken naar de huiswerkopdrachten. Die heb je ook heel erg goed ingevuld. Bij voorbaat al dank. We gaan het daar ook over hebben. Ja. Dus, goed dat we er zijn. Ik zal even een korte introductie doen. En dan kunnen we het gesprek induiken. Mijn naam is Michael. En ik werk voor een onderzoeksbureau. En we zijn nu onderzoek aan het doen. Je had het waarschijnlijk al door. Naar ervaringen rondom

In [3]:
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

LANG_ID = "nl"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-dutch"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
align_model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

In [4]:
device = 'cpu'
pipeline_type = "huggingface"
align_model = align_model.to(device)
labels = processor.tokenizer.get_vocab()
align_dictionary = {char.lower(): code for char,code in processor.tokenizer.get_vocab().items()}

align_metadata = {"language": result["language"], "dictionary": align_dictionary, "type": pipeline_type}

In [5]:
import whisperx

result_aligned = whisperx.align(result["segments"], align_model, align_metadata, SOURCE_FILE, device)

In [6]:
# Storing words <> timestamps mapping in a file.
import json

with open('./word_ts.text', 'w+') as f:
    for line in result_aligned['word_segments']:
        line_temp = line.copy()
        # WhisperX don't put a space after word but just to make sure.
        line_temp['text'] = line_temp['text'].strip()
        f.write(f'{json.dumps(line_temp)}\n')

In [21]:
# Settings for Nvidia
import os
import json

diarize_manifest = {
    'audio_filepath': f'./data/2min_reduced.wav',
    'offset': 0,
    'duration':  None,
    'label': "infer",
    'text': "-",
    'num_speakers': None,
    'rttm_filepath': f'./diarized/pred_rttms/audio_16k.rttm',
    'uniq_id': ""
}

if not os.path.exists('./manifest.json'):
    with open('./manifest.json', 'w') as f:
        f.write(json.dumps(diarize_manifest))

In [8]:
import os
import wget
from omegaconf import OmegaConf
MODEL_CONFIG = os.path.join('./','diar_infer_meeting.yaml')
if not os.path.exists(MODEL_CONFIG):
    config_url = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_meeting.yaml"
    MODEL_CONFIG = wget.download(config_url, './')

config = OmegaConf.load(MODEL_CONFIG)

In [22]:
config.num_workers = 0
config.batch_size = 32

config.diarizer.manifest_filepath = './manifest.json'
config.diarizer.out_dir = os.path.join('./', 'diarized')
config.diarizer.speaker_embeddings.model_path = 'titanet_large'
config.diarizer.speaker_embeddings.parameters.window_length_in_sec = [1.5, 1.0, 0.5]
config.diarizer.speaker_embeddings.parameters.shift_length_in_sec = [0.75, 0.5, 0.25]
config.diarizer.speaker_embeddings.parameters.multiscale_weights = [0.33, 0.33, 0.33]
config.diarizer.speaker_embeddings.parameters.save_embeddings = False

config.diarizer.ignore_overlap = False
config.diarizer.oracle_vad = False
config.diarizer.collar = 0.25


config.diarizer.vad.model_path = 'vad_multilingual_marblenet'
config.diarizer.oracle_vad = False # ----> Not using oracle VAD 

In [23]:
from nemo.collections.asr.models.msdd_models import ClusteringDiarizer

model = ClusteringDiarizer(cfg=config)

[NeMo I 2023-01-27 16:16:11 clustering_diarizer:129] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2023-01-27 16:16:11 cloud:56] Found existing object C:\Users\david\.cache\torch\NeMo\NeMo_1.14.0\vad_multilingual_marblenet\670f425c7f186060b7a7268ba6dfacb2\vad_multilingual_marblenet.nemo.
[NeMo I 2023-01-27 16:16:11 cloud:62] Re-using file from: C:\Users\david\.cache\torch\NeMo\NeMo_1.14.0\vad_multilingual_marblenet\670f425c7f186060b7a7268ba6dfacb2\vad_multilingual_marblenet.nemo
[NeMo I 2023-01-27 16:16:11 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2023-01-27 16:16:11 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2023-01-27 16:16:11 features:267] PADDING: 16
[NeMo I 2023-01-27 16:16:11 save_restore_connector:243] Model EncDecClassificationModel was successfully restored from C:\Users\david\.cache\torch\NeMo\NeMo_1.14.0\vad_multilingual_marblenet\670f425c7f186060b7a7268ba6dfacb2\vad_multilingual_marblenet.nemo.
[NeMo I 2023-01-27 16:16:11 clustering_diarizer:156] Loading pretrained titanet_large model from NGC
[NeMo I 2023-01-27 16:16:11 cloud:56] Found existing object C:\Users\david\.cache\torch\NeMo\NeMo_1.14.0\titanet-l\492c0ab8416139171dc18c21879a9e45\titanet-l.nemo.
[NeMo I 2023-01-27 16:16:11 cloud:62] Re-using file from: C:\Users\david\.cache\torch\NeMo\NeMo_1.14.0\titanet-l\492c0ab8416139171dc18c21879a9e45\titanet-l.nemo
[NeMo I 2023-01-27 16:16:11 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2023-01-27 16:16:11 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-01-27 16:16:11 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-01-27 16:16:11 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-01-27 16:16:11 features:267] PADDING: 16
[NeMo I 2023-01-27 16:16:12 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from C:\Users\david\.cache\torch\NeMo\NeMo_1.14.0\titanet-l\492c0ab8416139171dc18c21879a9e45\titanet-l.nemo.


In [24]:
model.diarize()

[NeMo I 2023-01-27 16:16:17 speaker_utils:92] Number of files to diarize: 1
[NeMo I 2023-01-27 16:16:17 clustering_diarizer:303] Split long audio file to avoid CUDA memory issue


splitting manifest: 100%|██████████| 1/1 [00:01<00:00,  1.76s/it]

[NeMo I 2023-01-27 16:16:19 vad_utils:100] The prepared manifest file exists. Overwriting!
[NeMo I 2023-01-27 16:16:19 classification_models:247] Perform streaming frame-level VAD
[NeMo I 2023-01-27 16:16:19 collections:296] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-01-27 16:16:19 collections:300] # 3 files loaded accounting to # 1 labels



vad:   0%|          | 0/3 [00:00<?, ?it/s][NeMo W 2023-01-27 16:16:19 nemo_logging:349] c:\src\interview-summary\venv\lib\site-packages\torch\amp\autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    
vad: 100%|██████████| 3/3 [00:22<00:00,  7.54s/it]

[NeMo I 2023-01-27 16:16:41 clustering_diarizer:258] Converting frame level prediction to speech/no-speech segment in start and end times format.


...

[NeMo I 2023-01-27 16:16:50 clustering_diarizer:281] Subsegmentation for embedding extraction: scale1, ./diarized\speaker_outputs\subsegments_scale1.json
[NeMo I 2023-01-27 16:16:50 clustering_diarizer:336] Extracting embeddings for Diarization
[NeMo I 2023-01-27 16:16:50 collections:296] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-01-27 16:16:50 collections:300] # 223 files loaded accounting to # 1 labels



[2/3] extract embeddings: 100%|██████████| 7/7 [00:08<00:00,  1.21s/it]

[NeMo I 2023-01-27 16:16:58 clustering_diarizer:281] Subsegmentation for embedding extraction: scale2, ./diarized\speaker_outputs\subsegments_scale2.json
[NeMo I 2023-01-27 16:16:58 clustering_diarizer:336] Extracting embeddings for Diarization
[NeMo I 2023-01-27 16:16:58 collections:296] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-01-27 16:16:58 collections:300] # 449 files loaded accounting to # 1 labels



[3/3] extract embeddings: 100%|██████████| 15/15 [00:10<00:00,  1.36it/s]
[NeMo W 2023-01-27 16:17:09 speaker_utils:428] cuda=False, using CPU for eigen decomposition. This might slow down the clustering process.
clustering: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]
[NeMo W 2023-01-27 16:17:11 der:105] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2023-01-27 16:17:11 clustering_diarizer:455] Outputs are saved in c:\src\interview-summary\diarized directory


In [26]:
# Reading timestamps <> Speaker Labels mapping

speaker_ts = []
with open('./diarized/pred_rttms/2min_reduced.rttm', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line_list = line.split(' ')
        s = int(float(line_list[5]) * 1000)
        e = s + int(float(line_list[8]) * 1000)
        speaker_ts.append([s, e, int(line_list[11].split('_')[-1])])

In [27]:
# Reading words <> timestamps mapping, which we saved earlier
import json

word_ts = []
with open('./word_ts.text', 'r+') as f:
    for line in f:
        line_temp = json.loads(line)
        word_ts.append(line_temp)

In [28]:
def get_word_ts_anchor(s, e, option='start'):
  if option == 'end':
    return e
  elif option == 'mid':
    return (s + e) / 2
  return s

def get_words_speaker_mapping(wrd_ts, spk_ts, word_anchor_option='start'):
    s, e, sp = spk_ts[0]
    wrd_pos, turn_idx = 0, 0
    wrd_spk_mapping = []
    for wrd_dict in wrd_ts:
        ws, we, wrd = int(wrd_dict['start'] * 1000), int(wrd_dict['end'] * 1000), wrd_dict['text']
        wrd_pos = get_word_ts_anchor(ws, we, word_anchor_option)
        while wrd_pos > float(e):
            turn_idx += 1
            turn_idx = min(turn_idx, len(spk_ts) - 1)
            s, e, sp = spk_ts[turn_idx]
        wrd_spk_mapping.append({'word': wrd, 'start_time': ws, 'end_time': we, 'speaker': sp})
    return wrd_spk_mapping

In [29]:
wsm = get_words_speaker_mapping(word_ts, speaker_ts, 'start')

In [40]:
current_speaker = 0
conversation = []
segment = {'speaker': current_speaker, 'text': []}
for word_dict in wsm:
    if word_dict['speaker'] != current_speaker:
        conversation.append(segment)
        current_speaker = 0 if current_speaker == 1 else 1
        segment = {'speaker': current_speaker, 'text': []}
    segment['text'].append(f"{word_dict['word']} ")
conversation.append(segment)

with open('./word_ts_final.txt', 'w+') as f:
    for segment in conversation:
        f.write(f"Speaker {segment['speaker']}\n")
        f.write(f"{''.join(segment['text'])}\n\n")